# Modelos Lineales

Los modelos lineales son una clase de modelos que se utilizan ampliamente en la practica y se han estudiado ampliamente en las ultimas decadas. Los modelos lineales hacen una prediccion utilizando una funcion lineal de las caracteristicas de entrada.